# OpenAI API 강의 
## 주제: Embedding과 벡터 기반 검색 시스템 구축


## 학습 목표
- Embedding(임베딩)의 개념 이해
- OpenAI Embedding API를 이용해 문장 벡터를 생성하는 방법 학습
- 문장 간 유사도 계산 실습
- 간단한 벡터 기반 검색 시스템 구현



## 1. Embedding이란?
| 항목 | 설명 |
|------|------|
| 정의 | 텍스트, 이미지, 음성 등 데이터를 벡터(숫자 배열)로 변환하는 과정 |
| 목적 | 의미가 비슷한 데이터끼리 벡터 공간에서 가깝게 배치 |
| 활용 분야 | 검색, 추천, 분류, RAG(Retrieval-Augmented Generation) |


### 예시 개념도
- 문장 A: '고양이는 귀엽다.' → [0.12, -0.53, 0.98, ...]
- 문장 B: '강아지는 사랑스럽다.' → [0.10, -0.50, 1.02, ...]

→ 두 벡터의 코사인 유사도가 높을수록 의미가 유사함

## 2. OpenAI Embedding API 사용하기

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    model="text-embedding-3-small",
    input="인공지능은 인간의 학습 능력을 모방합니다."
)
embedding_vector = response.data[0].embedding
print(len(embedding_vector))
print(embedding_vector[:10])  # 앞 10개 값 출력

1536
[-0.014983989298343658, 0.043729331344366074, -0.00992388091981411, 0.009673998691141605, 0.008527219295501709, -0.050297655165195465, 0.010022048838436604, 0.08017639070749283, 0.034894220530986786, 0.028825661167502403]



## 3. 문장 간 유사도 계산하기
- 코사인 유사도(Cosine Similarity) 사용
- 두 벡터의 방향이 얼마나 비슷한지를 측정 (값의 범위: -1 ~ 1)


In [2]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

emb1 = client.embeddings.create(input="인공지능의 정의", model="text-embedding-3-small").data[0].embedding
emb2 = client.embeddings.create(input="AI란 무엇인가?", model="text-embedding-3-small").data[0].embedding

print(emb1)
print(emb2)
similarity = cosine_similarity(emb1, emb2)
print("유사도:", similarity)


[-0.018507521599531174, 0.01572255976498127, -0.008365274406969547, 0.016896815970540047, 0.019141411408782005, -0.06555049866437912, -0.029491489753127098, 0.012469975277781487, 0.014007938094437122, 0.028078226372599602, 0.03304543346166611, 0.007928825914859772, -0.011482768692076206, -0.0647607371211052, 0.008879661560058594, -0.010568303987383842, -0.050170864909887314, -0.010204596444964409, 0.04809253662824631, -0.026706529781222343, 0.002436840906739235, 0.01855948008596897, -0.006692219991236925, 0.009513551369309425, 0.007014360744506121, -0.03433399647474289, 0.038656920194625854, 0.01840360462665558, 0.006697415839880705, -0.003870887914672494, 0.020253317430615425, -0.035996660590171814, 0.005850496701896191, -0.05062809959053993, 0.050586532801389694, 0.013238956220448017, 0.005840105004608631, 0.01360266376286745, 0.006629869807511568, 0.014548303559422493, -0.011358068324625492, 0.0318400040268898, 0.0038215278182178736, 0.04002862051129341, -0.007835300639271736, 0.037


## 4. 간단한 벡터 검색 시스템 만들기
1. 여러 문장을 벡터화하여 DB(혹은 리스트)에 저장
2. 사용자 입력 문장의 벡터를 생성
3. 기존 문장들과의 유사도 계산 후 상위 결과 반환


In [3]:

# 1) 문장 데이터
sentences = [
    "인공지능은 인간의 사고를 모방합니다.",
    "기계학습은 데이터로부터 패턴을 학습합니다.",
    "딥러닝은 인공신경망을 기반으로 합니다.",
    "자연어처리는 인간 언어를 이해하는 기술입니다."
]

# 2) 각 문장을 임베딩
embeddings = [client.embeddings.create(input=s, model="text-embedding-3-small").data[0].embedding for s in sentences]

# 3) 사용자 질의
query = "인공신경망 기반 학습 방법"
query_emb = client.embeddings.create(input=query, model="text-embedding-3-small").data[0].embedding

# 4) 유사도 계산
scores = [cosine_similarity(query_emb, e) for e in embeddings]

# 5) 상위 결과 출력
for idx, score in sorted(enumerate(scores), key=lambda x: x[1], reverse=True):
    print(f"{score:.3f} : {sentences[idx]}")


0.481 : 딥러닝은 인공신경망을 기반으로 합니다.
0.369 : 인공지능은 인간의 사고를 모방합니다.
0.341 : 기계학습은 데이터로부터 패턴을 학습합니다.
0.218 : 자연어처리는 인간 언어를 이해하는 기술입니다.



## 5. 실습 과제
1. 자신이 직접 5~10개의 문장을 준비하고 벡터 검색을 수행해보세요.
2. 검색 정확도를 높이기 위해 `text-embedding-3-large` 모델로 변경해 보세요.
3. 검색 결과를 시각화(matplotlib, seaborn 등)로 표현해 보세요.


In [4]:

response = client.embeddings.create(
    # model="text-embedding-3-small",
    model="text-embedding-3-large",
    input="인공지능은 인간의 학습 능력을 모방합니다."
)
embedding_vector = response.data[0].embedding
print(len(embedding_vector))
print(embedding_vector[:10])  # 앞 10개 값 출력

3072
[-0.019963255152106285, 0.0016327090561389923, -0.012867744080722332, -0.035911042243242264, 0.00811078492552042, 0.03082326613366604, 0.018719829618930817, 0.010603340342640877, 0.0063369073905050755, -0.002896097954362631]



## 참고 자료
- [OpenAI Embeddings API](https://platform.openai.com/docs/api-reference/embeddings)
- [Understanding Embeddings - OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/Embedding_long_inputs.ipynb)
- [Pinecone: RAG 기본 개념](https://www.pinecone.io/learn/retrieval-augmented-generation/)
